In [66]:
import numpy as np
from scipy.stats import geom, multivariate_normal

On fixe les paramètres.

In [67]:
N_sample = 1024
dim_x = 20
dim_z = 20
r = 0.6

On définit les fonctions utiles au projet.

In [ ]:
def p_theta_x(theta):
    mu = theta
    covariance = 2 * np.eye(dim_x)
    return multivariate_normal(mean=mu, cov=covariance)

def p_theta_x_given_z(z):
    mu = z
    covariance = np.eye(dim_x)
    return multivariate_normal(mean=mu, cov=covariance)

def p_theta_z(theta):
    mu = theta
    covariance = np.eye(dim_z)
    return multivariate_normal(mean=mu, cov=covariance)

p = geom

def logmeanexp(data, axis=None):
    max_val = np.max(data, axis=axis)
    return max_val + np.log(np.mean(np.exp(data - max_val), axis=axis))

On simule theta et les observations. On calcule la log-vraisemblance.

In [68]:
theta_0 = multivariate_normal.rvs(mean=np.zeros(dim_x), cov=np.eye(dim_x))
X = p_theta_x(theta_0).rvs(size=N_sample)
print(X[:2])

l_true = np.mean(np.log(p_theta_x(theta_0).pdf(X)))
print(l_true)

[[ 0.18807516  3.6219837  -0.05673272  2.55198827  2.38782273  3.25355657
  -0.00917009 -1.40009828  0.79325289 -0.99503646 -1.26296588  1.41633594
  -1.37389288  1.08842123 -0.6537514   0.30837661  0.06512314 -0.22977623
  -2.16468235  0.22783729]
 [-0.45688022  1.71982854 -1.29283098  1.28492627  1.92415116  0.49088385
  -0.3606232   0.41376882 -0.50955342 -3.57356841  2.61659233  0.0884764
   0.60850757 -0.12413669 -0.58052313  2.38607399 -1.05280288  0.23448334
   0.63669222 -0.773552  ]]
-35.35012983417641


On définit A et b.

In [69]:
A = 1 / 2 * np.eye(dim_z, dim_x) #size = dim_z, dim_x
b = np.mean(X, axis=0) #size = dim_z, 1

On peut donc maintenant définir l'encodeur.

In [70]:
def q_phi_z_given_x(x):
    mu = A @ x + b
    covariance = 2 / 3 * np.eye(dim_z)
    return multivariate_normal(mean=mu, cov=covariance)

On fait tourner l'algorithme décrit dans le papier pour chaque observation.

In [72]:
l_hat_ml_ss_list = []
l_hat_ml_rr_list = []

for x in X:
    K = p.rvs(r)
    size = 2 ** (K + 1)
    Z = q_phi_z_given_x(x).rvs(size=size)
    Z_E, Z_O = Z[::2], Z[1::2]
    log_weights = [np.log(p_theta_z(theta_0).pdf(z) * p_theta_x_given_z(z).pdf(x) / q_phi_z_given_x(x).pdf(z)) for z in Z]
    log_weights_E, log_weights_O = log_weights[::2], log_weights[1::2]
    I_0 = np.mean(log_weights)
    l_hat_E, l_hat_O = logmeanexp(log_weights_E), logmeanexp(log_weights_O)
    l_hat_O_E = logmeanexp(log_weights)
    delta_K = l_hat_O_E - 0.5 * (l_hat_O + l_hat_E)
    l_hat_ml_ss_x = I_0 + delta_K / p(r).pmf(K)
    l_hat_ml_ss_list.append(l_hat_ml_ss_x)
    delta_k_list = [logmeanexp(log_weights[:2**(k+1)]) - 0.5 * (logmeanexp(log_weights_O[:2**k]) + logmeanexp(log_weights_E[:2**k])) for k in range(K+1)]
    l_hat_ml_rr_x = I_0 + np.sum([delta_k_list[k] / (1 - p(r).cdf(k-1)) for k in range(K+1)])
    l_hat_ml_rr_list.append(l_hat_ml_rr_x)


On calcule les estimateurs finaux et les biais empiriques.

In [73]:
l_hat_ml_ss = np.mean(l_hat_ml_ss_list)
print("estimateur ss =", l_hat_ml_ss)

l_hat_ml_rr = np.mean(l_hat_ml_rr_list)
print("estimateur rr =", l_hat_ml_rr)

empirical_bias_squared_ss = (l_hat_ml_ss - l_true) ** 2
print("carré du biais empirique de l'estimateur ss :", empirical_bias_squared_ss)

empirical_bias_squared_rr = (l_hat_ml_rr - l_true) ** 2
print("carré du biais empirique de l'estimateur rr :", empirical_bias_squared_rr)

estimateur ss = -36.95528682955278
estimateur rr = -35.70554390511783
carré du biais empirique de l'estimateur ss : 2.5765289798057154
carré du biais empirique de l'estimateur rr : 0.12631916182315364
